In [1]:
import numpy as np
import xarray as xr
from datetime import datetime as dt

In [2]:
## Import data

## CAM Hindcast data will be imported into a list. Each index represents the number of hindcast day

# Setup strings to be used to create the path to each file
hcastDayStr = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']
# hcastDayStr = ['05','10','15','20']
fn = 'tratl_cam5.3_ne30_rneale.globe.fcast.day'
dri = '/home/jlarson1/data/CAM_hindcasts/'

# Initialize empty list that will contain all the specified hindcast days
dataPRECTRaw = [None] * (len(hcastDayStr)+1)
dataVBOT3hr  = [None] * (len(hcastDayStr)+1)
dataUBOT3hr  = [None] * (len(hcastDayStr)+1)

# Iterate through the string that dictates which hindcast days will be loaded
for ii,day in enumerate(hcastDayStr):
#     print(dri+fn+day+'.PRECT.nc')
    dataPRECTRaw[ii+1] = xr.open_dataset(dri+fn+day+'.PRECT.nc')
    dataVBOT3hr[ii+1]  = xr.open_dataset(dri+fn+day+'.VBOT.nc')
    dataUBOT3hr[ii+1]  = xr.open_dataset(dri+fn+day+'.UBOT.nc')
    
## Observation precipitation

# driObs = '/lss/research/agon-lab/TMPA-daily/'
driPRECTObs = '/home/jlarson1/data/'
fnPRECTObs  = 'pr_daily_2009-2010.nc'
dataPRECTObs = xr.open_dataset(driPRECTObs + fnPRECTObs)

## Observation surface winds

# Note that these datasets were created specifically for this set of hindcasts. 
# For future reference, if a more generic dataset(s) is used, the code in the next few sections might be useful
driWindObs = '/home/jlarson1/data/ERA5/'
fnUBOTObs = 'u10m_2009-2010-partial_daily.nc' ; fnVBOTObs = 'v10m_2009-2010-partial_daily.nc'
dataUBOTObs = xr.open_dataset(driWindObs + fnUBOTObs)
dataVBOTObs = xr.open_dataset(driWindObs + fnVBOTObs)

In [4]:
# Convert time stamps from cftime to datetime format

# Initialize array to hold every day's (152 days) time stamp for each hindcast day (20 days)
prectTime = np.array([ [None] * len(dataPRECTRaw[1]['time']) ] * len(dataPRECTRaw))

# Loop through all hindcast days
# Start the index at 1 so the indexing is more intuitive (e.g. day 7 hindcast is indexed with a 7)
for ii in range(1,  len(dataPRECTRaw) ):
    # Loop through each day in the hindcast dataset
    for jj in range( len(dataPRECTRaw[1]['time']) ):
        # Parse through the time stamp and convert to the datetime class
        prectTime[ii][jj] = dt.strptime(str(dataPRECTRaw[ii]['time'][jj].values), '%Y-%m-%d %H:%M:%S')

In [5]:
# # Select the obs data from the years of interest (2009 and 2010) that is more or less within the hindcast time bounds
# # Concat together. All this is done to reduce the amount of data and speed the code

# hcastTimeStart = 0   # days since 2009-09-01
# _hcastTimeEnd  = 200 # days since 2009-09-01

# # Adjust end time to fit format of obs data which ends with the decimal .75
# hcastTimeEnd = _hcastTimeEnd - 0.25

# # Select the data that is at the end of 2009 and beginning of 2010 along the time dimension
# dataVBOTObs = xr.concat([dataVBOTObs2009.sel(time=slice((hcastTimeStart+40055)*24,None)),
#                          dataVBOTObs2010.sel(time=slice((hcastTimeEnd+40055)*24))], dim='time')
# dataUBOTObs = xr.concat([dataUBOTObs2009.sel(time=slice((hcastTimeStart+40055)*24,None)),
#                          dataUBOTObs2010.sel(time=slice((hcastTimeEnd+40055)*24))], dim='time')

In [6]:
# # # Time average the 6-hourly wind obs data to get daily wind data

# # Create hindcast day dimension
# hcastDay = [5, 10, 15, 20]

# dataVBOTObsDaily = np.zeros(( len(dataVBOTObs['v10'])//4, len(dataVBOTObs['v10'][0]), len(dataVBOTObs['v10'][0][0]) ))
# dataUBOTObsDaily = np.zeros(( len(dataUBOTObs['u10'])//4, len(dataUBOTObs['u10'][0]), len(dataUBOTObs['u10'][0][0]) ))
# WindObsTime    = np.zeros(( len(dataVBOTObs['v10'])//4 ))

# # Iterate for the number of time stamps in the 6hrly data. Count by 4
# for ii in range (0, len(dataVBOTObs['v10']), 4):
#     dataVBOTObsDaily[ii//4] = dataVBOTObs['v10' ][ii:ii+4].mean(axis=0)
#     dataUBOTObsDaily[ii//4] = dataUBOTObs['u10' ][ii:ii+4].mean(axis=0)
#     # Create new time stamps as "days since 2009-09-01"
#     WindObsTime[ii//4]    = dataVBOTObs['time'][ii:ii+4].mean(axis=0)//24 - 40055

In [7]:
# Time average the 3-hourly wind hindcast data to get daily wind data

# Create an empty array to hold the daily averaged data
vbot = np.zeros([len(hcastDayStr)+1, len(dataVBOT3hr[1]['time'])//8, len(dataVBOT3hr[1]['lat']), len(dataVBOT3hr[1]['lon'])])
ubot = np.zeros([len(hcastDayStr)+1, len(dataUBOT3hr[1]['time'])//8, len(dataUBOT3hr[1]['lat']), len(dataUBOT3hr[1]['lon'])])

# Iterate for the number of hindcast days
for ii in range(1, len(vbot) ):

    # Iterate for the number of time stamps in 3hrly data. Count by 8
    for jj in range(0, len(dataVBOT3hr[1]['V']), 8):
        # On the left, jj is divided by 8 so it increases by 1 (each day)
        # On the right, 8 is added to jj to include all 8 daily data points in the mean
        vbot[ii][jj//8] = dataVBOT3hr[ii]['V'][jj:jj+8].mean(dim='time').values
        ubot[ii][jj//8] = dataUBOT3hr[ii]['U'][jj:jj+8].mean(dim='time').values
        

In [8]:
# Organize the daily horizontal wind data into a dataset
# Also, recreate the precip datasets with the new datetime-formatted time stamps

# Initialize empty arrays to hold the new dataArrays
dataVBOT  = [None] * (len(hcastDayStr)+1)
dataUBOT  = [None] * (len(hcastDayStr)+1)
dataPRECT = [None] * (len(hcastDayStr)+1)

# Loop through list and create dataArrays
for ii in range(1, len(vbot) ):
    dataVBOT[ii]  = xr.DataArray(vbot[ii], dims=['time','lat','lon'], 
                                coords={'time':prectTime[ii], 'lat':dataVBOT3hr[1]['lat'], 'lon':dataVBOT3hr[1]['lon']})
    
    dataUBOT[ii]  = xr.DataArray(ubot[ii], dims=['time','lat','lon'],
                                coords={'time':prectTime[ii], 'lat':dataUBOT3hr[1]['lat'], 'lon':dataUBOT3hr[1]['lon']})
    
    # Convert precip from m/s to mm/day
    dataPRECT[ii] = xr.DataArray(dataPRECTRaw[ii]['PRECT']*1000*86400, dims=['time','lat','lon'],
                                coords={'time':prectTime[ii], 'lat':dataPRECTRaw[1]['lat'], 'lon':dataPRECTRaw[1]['lon']})